# PACKAGES AND MODULES INSTALLATION AND IMPORTING

In [ ]:
!pip install duckling
!pip install language
!pip install dim
! conda install -y gdown 
!pip install Jpype1==0.7.5

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# DATA IMPORTING

In [ ]:
!gdown --id 11pTm8RHNk8mMtgmbHndygZINdwrojS7R

import pandas as pd
data=pd.read_json("./data_small.json")

## DECLARATION OF DIM AND LANGUAGE FOR DUCKLING

In [ ]:
class Dim(object):

    """Helper class containing all supported dimensions
    """

    AMOUNTOFMONEY = u'amount-of-money'
    CYCLE = u'cycle'
    DISTANCE = u'distance'
    DURATION = u'duration'
    EMAIL = u'email'
    LEVENPRODUCT = u'leven-product'
    LEVENUNIT = u'leven-unit'
    NUMBER = u'number'
    ORDINAL = u'ordinal'
    PHONENUMBER = u'phone-number'
    QUANTITY = u'quantity'
    TEMPERATURE = u'temperature'
    TIME = u'time'
    TIMEZONE = u'timezone'
    UNIT = u'unit'
    UNITOFDURATION = u'unit-of-duration'
    URL = u'url'
    VOLUME = u'volume'

class Language(object):

    """Helper class containing all supported languages
    """

    ARABIC = 'ar$core'
    DANISH = 'da$core'
    GERMAN = 'de$core'
    ENGLISH = 'en$core'
    SPANISH = 'es$core'
    ESTONIAN = 'et$core'
    FRENCH = 'fr$core'
    IRISH = 'ga$core'
    INDONESIAN = 'id$core'
    ITALIAN = 'it$core'
    JAPANESE = 'ja$core'
    KOREAN = 'ko$core'
    PORTUGUESE = 'pt$core'
    RUSSIAN = 'ru$core'
    UKRAINIAN = 'uk$core'
    VIETNAMESE = 'vi$core'
    CHINESE = 'zh$core'

    SUPPORTED_LANGUAGES = {
        ARABIC,
        DANISH,
        GERMAN,
        ENGLISH,
        SPANISH,
        ESTONIAN,
        FRENCH,
        IRISH,
        INDONESIAN,
        ITALIAN,
        JAPANESE,
        KOREAN,
        PORTUGUESE,
        RUSSIAN,
        UKRAINIAN,
        VIETNAMESE,
        CHINESE,
    }

    @classmethod
    def is_supported(cls, lang):
        """Check if a language is supported by the current duckling version."""

        return lang in cls.SUPPORTED_LANGUAGES

    @classmethod
    def convert_to_duckling_language_id(cls, lang):
        """Ensure a language identifier has the correct duckling format and is supported."""

        if lang is not None and cls.is_supported(lang):
            return lang
        elif lang is not None and cls.is_supported(lang + "$core"):   # Support ISO 639-1 Language Codes (e.g. "en")
            return lang + "$core"
        else:
            raise ValueError("Unsupported language '{}'. Supported languages: {}".format(
                lang, ", ".join(cls.SUPPORTED_LANGUAGES)))

    @classmethod
    def convert_to_iso(cls, lang):
        return cls.convert_to_duckling_language_id(lang)[:2]

# DECLARATION FOR DUCKLING

In [ ]:
from duckling import DucklingWrapper
from duckling import Duckling



class DucklingWrapper(object):

    """Simplified Python wrapper for Duckling by wit.ai.
    Attributes:
        jvm_started: Optional attribute to specify if the JVM has already been
            started (with all Java dependencies loaded).
        parse_datetime: Optional attribute to specify if datetime string should
            be parsed with datetime.strptime(). Default is False.
        language: Optional attribute to specify language to be used with
            Duckling. Default is Language.ENGLISH. E.g. Duckling.GERMAN or supported ISO 639-1 Code (e.g. "de")
        minimum_heap_size: Optional attribute to set initial and minimum heap
            size. Default is 128m.
        maximum_heap_size: Optional attribute to set maximum heap size. Default
            is 2048m.
    """

    def __init__(self,
                 jvm_started=False,
                 parse_datetime=False,
                 language=Language.ENGLISH,
                 minimum_heap_size='128m',
                 maximum_heap_size='2048m'):
        super(DucklingWrapper, self).__init__()
        self.language = Language.convert_to_duckling_language_id(language)
        self.duckling = Duckling(
            jvm_started=jvm_started,
            parse_datetime=parse_datetime,
            minimum_heap_size=minimum_heap_size,
            maximum_heap_size=maximum_heap_size)
        self.duckling.load()
        self._dims = {
            Dim.TIME:           self._parse_time,
            Dim.TEMPERATURE:    self._parse_number_with_unit,
            Dim.NUMBER:         self._parse_basic_info,
            Dim.ORDINAL:        self._parse_basic_info,
            Dim.DISTANCE:       self._parse_number_with_unit,
            Dim.VOLUME:         self._parse_volume,
            Dim.AMOUNTOFMONEY:  self._parse_number_with_unit,
            Dim.DURATION:       self._parse_duration,
            Dim.EMAIL:          self._parse_basic_info,
            Dim.URL:            self._parse_basic_info,
            Dim.PHONENUMBER:    self._parse_basic_info,
            Dim.TIMEZONE:       self._parse_basic_info,
            Dim.LEVENPRODUCT:   self._parse_basic_info,
            Dim.LEVENUNIT:      self._parse_basic_info,
            Dim.QUANTITY:       self._parse_quantity,

            Dim.CYCLE:          self._parse_basic_info,
            Dim.UNIT:           self._parse_basic_info,
            Dim.UNITOFDURATION: self._parse_basic_info

        }

    def _parse(self, input_str, dim=None, reference_time=''):
        result = []
        duckling_result = self.duckling.parse(
            input_str, self.language, dim_filter=dim,
            reference_time=reference_time)
        for entry in duckling_result:
            if entry[u'dim'] in self._dims:
                result_entry = self._dims[entry[u'dim']](entry)
                result.append(result_entry)
        return result

    def _parse_basic_info(self, duckling_result_entry):
        result_entry = {
            u'dim': duckling_result_entry[u'dim'],
            u'text': duckling_result_entry[u'body'],
            u'start': duckling_result_entry[u'start'],
            u'end': duckling_result_entry[u'end'],
            u'value': {
                u'value': duckling_result_entry[u'value'].get(u'value', None)
            }
        }
        if u'grain' in duckling_result_entry[u'value']:
            result_entry[u'value'][u'grain'] = duckling_result_entry[u'value'].get(u'grain', None)
        return result_entry

    def _parse_number_with_unit(self, duckling_result_entry):
        result_entry = self._parse_basic_info(duckling_result_entry)
        result_entry[u'value'].update({
            u'unit': duckling_result_entry[u'value'].get(u'unit', None)
        })
        return result_entry

    def _is_interval(self, duckling_result_entry):
        return u'from' in duckling_result_entry[u'value'] or \
            u'to' in duckling_result_entry[u'value']

    def _parse_interval(self, result_entry, duckling_result_entry):
        result_entry[u'value'].update({
            u'value': {
                u'to': duckling_result_entry[u'value'].get(u'to', {}).get(u'value', None),
                u'from': duckling_result_entry[u'value'].get(u'from', {}).get(u'value', None)
            },
            u'others': []
        })
        for value in duckling_result_entry[u'value'][u'values']:
            result_entry[u'value'][u'others'].append({
                u'to': value.get(u'to', {}).get(u'value', None),
                u'from': value.get(u'from', {}).get(u'value', None)
            })
        return result_entry

    def _parse_time(self, duckling_result_entry):
        result_entry = self._parse_basic_info(duckling_result_entry)
        if self._is_interval(duckling_result_entry):
            return self._parse_interval(result_entry, duckling_result_entry)
        result_entry[u'value'].update({
            u'others': []
        })
        for value in duckling_result_entry[u'value'][u'values']:
            result_entry[u'value'][u'others'].append({
                u'grain': value[u'grain'],
                u'value': value[u'value']
            })
        return result_entry

    def _parse_volume(self, duckling_result_entry):
        result_entry = self._parse_basic_info(duckling_result_entry)
        result_entry[u'value'].update({
            u'unit': duckling_result_entry[u'value'][u'unit']
            if u'unit' in duckling_result_entry[u'value'] else None,
            u'latent': duckling_result_entry[u'latent']
            if u'latent' in duckling_result_entry else False
        })
        return result_entry

    def _parse_duration(self, duckling_result_entry):
        result_entry = self._parse_basic_info(duckling_result_entry)
        result_entry[u'value'].update({
            u'unit': duckling_result_entry[u'value'][u'unit']
            if u'unit' in duckling_result_entry[u'value'] else None,
            u'year': duckling_result_entry[u'value'][u'year']
            if u'year' in duckling_result_entry[u'value'] else None,
            u'month': duckling_result_entry[u'value'][u'month']
            if u'month' in duckling_result_entry[u'value'] else None,
            u'day': duckling_result_entry[u'value'][u'day']
            if u'day' in duckling_result_entry[u'value'] else None,
            u'hour': duckling_result_entry[u'value'][u'hour']
            if u'hour' in duckling_result_entry[u'value'] else None,
            u'minute': duckling_result_entry[u'value'][u'minute']
            if u'minute' in duckling_result_entry[u'value'] else None,
            u'second': duckling_result_entry[u'value'][u'second']
            if u'second' in duckling_result_entry[u'value'] else None
        })
        return result_entry

    def _parse_quantity(self, duckling_result_entry):
        result_entry = self._parse_basic_info(duckling_result_entry)
        result_entry[u'value'].update({
            u'unit': duckling_result_entry[u'value'][u'unit']
            if u'unit' in duckling_result_entry[u'value'] else None,
            u'product': duckling_result_entry[u'value'][u'product']
            if u'product' in duckling_result_entry[u'value'] else None
        })
        return result_entry

    # Public API

    def parse(self, input_str, reference_time=''):
        """Parses input with Duckling for all dims.
        Args:
            input_str: An input string, e.g. 'You owe me twenty bucks, please
                call me today'.
            reference_time: Optional reference time for Duckling.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"number",
                  "end":17,
                  "start":11,
                  "value":{
                     "value":20.0
                  },
                  "text":"twenty"
               },
               {
                  "dim":"time",
                  "end":45,
                  "start":40,
                  "value":{
                     "value":"2016-10-10T00:00:00.000-07:00",
                     "others":[
                        "2016-10-10T00:00:00.000-07:00"
                     ]
                  },
                  "text":"today"
               },
               {
                  "dim":"amount-of-money",
                  "end":23,
                  "start":11,
                  "value":{
                     "unit":null,
                     "value":20.0
                  },
                  "text":"twenty bucks"
               },
               {
                  "dim":"distance",
                  "end":17,
                  "start":11,
                  "value":{
                     "unit":null,
                     "value":20.0
                  },
                  "text":"twenty"
               },
               {
                  "dim":"volume",
                  "end":17,
                  "start":11,
                  "value":{
                     "latent":true,
                     "unit":null,
                     "value":20.0
                  },
                  "text":"twenty"
               },
               {
                  "dim":"temperature",
                  "end":17,
                  "start":11,
                  "value":{
                     "unit":null,
                     "value":20.0
                  },
                  "text":"twenty"
               },
               {
                  "dim":"time",
                  "end":17,
                  "start":11,
                  "value":{
                     "value":"2020-01-01T00:00:00.000-08:00",
                     "others":[
                        "2020-01-01T00:00:00.000-08:00"
                     ]
                  },
                  "text":"twenty"
               }
            ]
        """
        return self._parse(input_str, reference_time=reference_time)

    def parse_time(self, input_str, reference_time=''):
        """Parses input with Duckling for occurences of times.
        Args:
            input_str: An input string, e.g. 'Let's meet at 11:45am'.
            reference_time: Optional reference time for Duckling.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"time",
                  "end":21,
                  "start":11,
                  "value":{
                     "value":"2016-10-11T11:45:00.000-07:00",
                     "others":[
                        "2016-10-11T11:45:00.000-07:00",
                        "2016-10-12T11:45:00.000-07:00",
                        "2016-10-13T11:45:00.000-07:00"
                     ]
                  },
                  "text":"at 11:45am"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.TIME,
                           reference_time=reference_time)

    def parse_timezone(self, input_str):
        """Parses input with Duckling for occurences of timezones.
        Args:
            input_str: An input string, e.g. 'My timezone is pdt'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"timezone",
                  "end":18,
                  "start":15,
                  "value":{
                     "value":"PDT"
                  },
                  "text":"pdt"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.TIMEZONE)

    def parse_temperature(self, input_str):
        """Parses input with Duckling for occurences of temperatures.
        Args:
            input_str: An input string, e.g. 'Let's change the temperature from
                thirty two celsius to 65 degrees'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"temperature",
                  "end":65,
                  "start":55,
                  "value":{
                     "unit":"degree",
                     "value":65.0
                  },
                  "text":"65 degrees"
               },
               {
                  "dim":"temperature",
                  "end":51,
                  "start":33,
                  "value":{
                     "unit":"celsius",
                     "value":32.0
                  },
                  "text":"thirty two celsius"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.TEMPERATURE)

    def parse_number(self, input_str):
        """Parses input with Duckling for occurences of numbers.
        Args:
            input_str: An input string, e.g. 'I'm 25 years old'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"number",
                  "end":6,
                  "start":4,
                  "value":{
                     "value":25.0
                  },
                  "text":"25"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.NUMBER)

    def parse_ordinal(self, input_str):
        """Parses input with Duckling for occurences of ordinals.
        Args:
            input_str: An input string, e.g. 'I'm first, you're 2nd'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"ordinal",
                  "end":9,
                  "start":4,
                  "value":{
                     "value":1
                  },
                  "text":"first"
               },
               {
                  "dim":"ordinal",
                  "end":21,
                  "start":18,
                  "value":{
                     "value":2
                  },
                  "text":"2nd"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.ORDINAL)

    def parse_distance(self, input_str):
        """Parses input with Duckling for occurences of distances.
        Args:
            input_str: An input string, e.g. 'I commute 5 miles everyday'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"distance",
                  "end":17,
                  "start":10,
                  "value":{
                     "unit":"mile",
                     "value":5.0
                  },
                  "text":"5 miles"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.DISTANCE)

    def parse_volume(self, input_str):
        """Parses input with Duckling for occurences of volumes.
        Args:
            input_str: An input string, e.g. '1 gallon is 3785ml'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"volume",
                  "end":18,
                  "start":12,
                  "value":{
                     "latent":false,
                     "unit":"millilitre",
                     "value":3785.0
                  },
                  "text":"3785ml"
               },
               {
                  "dim":"volume",
                  "end":8,
                  "start":0,
                  "value":{
                     "latent":false,
                     "unit":"gallon",
                     "value":1.0
                  },
                  "text":"1 gallon"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.VOLUME)

    def parse_money(self, input_str):
        """Parses input with Duckling for occurences of moneys.
        Args:
            input_str: An input string, e.g. 'You owe me 10 dollars'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"amount-of-money",
                  "end":21,
                  "start":11,
                  "value":{
                     "unit":"$",
                     "value":10.0
                  },
                  "text":"10 dollars"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.AMOUNTOFMONEY)

    def parse_duration(self, input_str):
        """Parses input with Duckling for occurences of durations.
        Args:
            input_str: An input string, e.g. 'I ran for 2 hours today'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"duration",
                  "end":17,
                  "start":10,
                  "value":{
                     "hour":2,
                     "value":2.0,
                     "month":null,
                     "second":null,
                     "minute":null,
                     "year":null,
                     "day":null,
                     "unit":"hour"
                  },
                  "text":"2 hours"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.DURATION)

    def parse_email(self, input_str):
        """Parses input with Duckling for occurences of emails.
        Args:
            input_str: An input string, e.g. 'Shoot me an email at
                contact@frank-blechschmidt.com'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"email",
                  "end":51,
                  "start":21,
                  "value":{
                     "value":"contact@frank-blechschmidt.com"
                  },
                  "text":"contact@frank-blechschmidt.com"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.EMAIL)

    def parse_url(self, input_str):
        """Parses input with Duckling for occurences of urls.
        Args:
            input_str: An input string, e.g. 'http://frank-blechschmidt.com is
                under construction, but you can check my github
                github.com/FraBle'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"url",
                  "end":98,
                  "start":81,
                  "value":{
                     "value":"github.com/FraBle"
                  },
                  "text":"github.com/FraBle"
               },
               {
                  "dim":"url",
                  "end":29,
                  "start":0,
                  "value":{
                     "value":"http://frank-blechschmidt.com"
                  },
                  "text":"http://frank-blechschmidt.com"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.URL)

    def parse_phone_number(self, input_str):
        """Parses input with Duckling for occurences of phone numbers.
        Args:
            input_str: An input string, e.g. '424-242-4242 is obviously a fake
                number'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim":"phone-number",
                  "end":13,
                  "start":0,
                  "value":{
                     "value":"424-242-4242 "
                  },
                  "text":"424-242-4242 "
               }
            ]
        """
        return self._parse(input_str, dim=Dim.PHONENUMBER)

    def parse_leven_product(self, input_str):
        """Parses input with Duckling for occurences of products.
        Args:
            input_str: An input string, e.g. '5 cups of sugar'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim": "leven-product",
                  "end": 15,
                  "start": 10,
                  "value":
                  {
                     "value": "sugar"
                  },
                  "text": "sugar"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.LEVENPRODUCT)

    def parse_leven_unit(self, input_str):
        """Parses input with Duckling for occurences of leven units.
        Args:
            input_str: An input string, e.g. 'two pounds of meat'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim": "leven-unit",
                  "start": 4,
                  "end": 10,
                  "value":
                  {
                     "value": "pound"
                  },
                  "text": "pounds"
               }
            ]
        """
        return self._parse(input_str, dim=Dim.LEVENUNIT)

    def parse_quantity(self, input_str):
        """Parses input with Duckling for occurences of quantities.
        Args:
            input_str: An input string, e.g. '5 cups of sugar'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output. For
            example:
            [
               {
                  "dim": "quantity",
                  "text": "5 cups of sugar",
                  "start": 0,
                  "end": 15,
                  "value":
                  {
                     "value": 5,
                     "unit": "cup",
                     "product": "sugar"
                  }
                }
            ]
        """
        return self._parse(input_str, dim=Dim.QUANTITY)

    def parse_cycle(self, input_str):
        """Parses input with Duckling for occurences of cycles.
        Args:
            input_str: An input string, e.g. 'coming week'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output.
        """
        return self._parse(input_str, dim=Dim.CYCLE)

    def parse_unit(self, input_str):
        """Parses input with Duckling for occurences of units.
        Args:
            input_str: An input string, e.g. '6 degrees outside'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output.
        """
        return self._parse(input_str, dim=Dim.UNIT)

    def parse_unit_of_duration(self, input_str):
        """Parses input with Duckling for occurences of units of duration.
        Args:
            input_str: An input string, e.g. '1 second'.
        Returns:
            A preprocessed list of results (dicts) from Duckling output.
        """
        return self._parse(input_str, dim=Dim.UNITOFDURATION)

In [ ]:
d=DucklingWrapper()

## THATS HOW IT WORKS

In [ ]:

a=d.parse_money(u'i have spend Rs 10.00 ')
a

# DATA FILTERING INVOLVING TRANSACTIONS

In [ ]:
filter_Data=data.loc[data['message'].str.contains("Rs")]

In [ ]:
#double filtering with keywords as well as using Duckling
filter_Data=filter_Data.loc[filter_Data['message'].str.contains("debited" or"spend" or "charged" or "paid" or "sent" or "txn" or "billed" or "transaction" or "credited") ]


## DATA OF A PARTICULAR USER

In [ ]:
#data of 1 user id
user1_data=filter_Data[filter_Data.user_id=="40ec6ba2-af1b-4037-9e01-3d108faa937b"]
#sort values by timestamp
user1_data=user1_data.sort_values("timestamp")
#taking month as 03 of year 2021
user1_data=user1_data[user1_data.timestamp.astype(str).str.contains("2021-03-")]

In [ ]:
#transactions for subscribed member
#make graphs for expenses
#famous bank HDFC 
#timestamp kis time range mai maximum transactions 
# No use - okay created_at
for i,rows in user1_data.iterrows():
    l=rows.message.find("Rs.")
    if l!=-1:
        user1_data.loc[i,["message"]]=rows.message[:l+3]+" "+rows.message[l+3:]
    print(user1_data.loc[i,["message"]])

In [ ]:
user1_data

In [ ]:
##find the data with debited balance
user1_debited_data=user1_data.loc[filter_Data['message'].str.contains("debited")]
user1_debited_array=np.array([])
user1_amount=np.array([])
user1_debited_amount=0
for i,rows in user1_debited_data.iterrows():
    a=d.parse_money(rows.message)
    if(len(a)):
        user1_debited_amount+=a[0]['value']['value']
        user1_debited_array=np.append(user1_debited_array,user1_debited_amount)
        user1_amount=np.append(user1_amount,a[0]['value']['value'])
    else:
        user1_debited_array=np.append(user1_debited_array,0)
        user1_amount=np.append(user1_amount,0)


In [ ]:
user1_debited_data["expense"]=user1_debited_array
user1_debited_data["amount"]=user1_amount

In [ ]:
user1_debited_data

In [ ]:
# find the data with credited balance
user1_credited_data=user1_data.loc[filter_Data['message'].str.contains("credited")]
user1_credited_amount=0
for i,rows in user1_credited_data.iterrows():
    a=d.parse_money(rows.message)
    if(len(a)):
        user1_credited_amount+=a[0]['value']['value']
user1_credited_data

# INSIGHTS OF DATA

MOST FAMOUS BANK

In [ ]:
filter_Data["sender"].value_counts()

**sender - ADHDFCBK**

Number of Transactions 

In [ ]:
#data of 1 user id
user1_data=filter_Data[filter_Data.user_id=="40ec6ba2-af1b-4037-9e01-3d108faa937b"]
#sort values by timestamp
user1_data=user1_data.sort_values("timestamp")


In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(20, 20), dpi=80)
import pandas as pd
import numpy as np
data_csv=user1_data
data1=data_csv.copy(deep=True)
data2=np.unique(data1['sender'])
data1['sender']=data1['sender'].astype('object')
data3=data1['sender'].value_counts()
plt.pie(data3,labels=data3.index)

# plot of number of transactions at particular amount

In [ ]:
import matplotlib.pyplot as plt

figure(figsize=(10, 10), dpi=80)
import pandas as pd
data_csv=pd.read_csv('graph.csv') 
data1=data_csv.copy(deep=True)
print(data1.columns)
data_ex=pd.crosstab(index=data1['expense'],columns=data1['sender'],dropna=True)
data_ex=data_ex.iloc[1:]
data_re=data_ex.sum(axis=0)
dat=data_re.index
data_amm=data1['amount']
data_amm=data_amm[data_amm!=0]
data_ew=pd.crosstab(index=data1['amount'],columns=data1['sender'],dropna=True)
data_n=data_ew.index
for i in range(0,len(dat),1):
    data_ew[dat[i]]=data_n*data_ew[dat[i]]
data_ew=data_ew.sum()
data_re=data_re[data_re!=0]
data_ew=data_ew[data_ew!=0]
plt.scatter(data_ew,data_re)

In [ ]:
##find the data with debited balance
user1_debited_data=user1_data.loc[filter_Data['message'].str.contains("debited")]

user1_debited_array=np.array([])
user1_amount=np.array([])
user1_debited_amount=0
for i,rows in user1_debited_data.iterrows():
    a=d.parse_money(rows.message)
    if(len(a)):
        user1_debited_amount+=a[0]['value']['value']
        user1_debited_array=np.append(user1_debited_array,user1_debited_amount)
        user1_amount=np.append(user1_amount,a[0]['value']['value'])
    else:
        user1_debited_array=np.append(user1_debited_array,0)
        user1_amount=np.append(user1_amount,0)
        
user1_debited_data["expense"]=user1_debited_array
user1_debited_data["amount"]=user1_amount

In [ ]:
user1_debited_data['expense']=user1_debited_data['expense'].replace(to_replace=0, method='ffill')
user1_debited_data.to_csv("graph.csv")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
data_csv=pd.read_csv('graph.csv') 
data1=data_csv.copy(deep=True)
data1.sort_values("timestamp")

plt.plot(data1["timestamp"],data1['expense'])
plt.title('time Vs expense')
plt.xlabel('time')
plt.ylabel('expense')


In [ ]:
plt.plot(data1["timestamp"],data1['amount'])
plt.title('time Vs amount')
plt.xlabel('time')
plt.ylabel('amount')